# Lab 12-5 sequence to sequence (Keras + eager version)

### simple neural machine translation training

* sequence to sequence
  
### Reference
* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)

In [4]:
pip install tensorflow==1.15

Note: you may need to restart the kernel to use updated packages.


Exception:
Traceback (most recent call last):
  File "C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 179, in main
    status = self.run(options, args)
  File "C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\pip\_internal\commands\install.py", line 315, in run
    resolver.resolve(requirement_set)
  File "C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\pip\_internal\resolve.py", line 131, in resolve
    self._resolve_one(requirement_set, req)
  File "C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\pip\_internal\resolve.py", line 294, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\pip\_internal\resolve.py", line 242, in _get_abstract_dist_for
    self.require_hashes
  File "C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\pip\_internal\operations\prepare.py", line 334, in prepare_linked_requirement
    progress_bar=self.progress_bar
  File "C:\User

In [55]:
pip install -U tensorflow

  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os

#요약
from gensim.summarization.summarizer import summarize




2.0.0


AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd
path = '/content/drive/My Drive/Colab Notebooks/moviedata2.json'
df = pd.read_json('moviedata2.json')[:10000]


In [20]:
df

,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
0,"[Adventure, Drama, Horror, Thriller]",47 Meters Down: Uncaged혻,www.imdb.com/title/tt7329656/mediaviewer/rm242...,[Four teen girls diving in a ruined underwater...,"[The film starts at a girls school in Yucatan,...","['\n', '\nThe Next Chapter', '\n']"
1,"[Drama, Fantasy, Horror]",It Chapter Two혻,www.imdb.com/title/tt7349950/mediaviewer/rm324...,[Twenty-seven years after their first encounte...,"[In 2016 Derry, Maine, a young gay man is beat...","['\n', ""\nYou'll Float Again"", '\n']"
2,"[Horror, Sci-Fi]",Alien혻,www.imdb.com/title/tt0078748/mediaviewer/rm299...,[After a space merchant vessel perceives an un...,[The opening credits appear in front of a larg...,"['\n', ""\nIt's Alien, the 8th passenger."", '\n']"
3,"[Horror, Mystery, Thriller]",The Conjuring 3혻,www.imdb.com/title/tt7069210/mediaviewer/rm175...,[],[],"['\n', '\n', '\xa0', '\n', '\xa0', '\n', '\n']"
4,"[Adventure, Drama, Fantasy]",The Lord of the Rings: The Fellowship of the R...,www.imdb.com/title/tt0120737/mediaviewer/rm359...,[A meek Hobbit from the Shire and eight compan...,"[The prologue, spoken by Galadriel, shows the ...","['\n', '\nAll we have to decide is what to do ..."
5,"[Crime, Drama, Thriller]",The Mule혻,www.imdb.com/title/tt7959026/mediaviewer/rm320...,[A 90-year-old horticulturist and Korean War v...,[This movie is based on a true story of a Worl...,"['\n', '\nNobody Runs Forever', '\n']"
6,"[Adventure, Family, Fantasy]",Harry Potter and the Sorcerer's Stone혻,www.imdb.com/title/tt0241527/mediaviewer/rm683...,[An orphaned boy enrolls in a school of wizard...,"[Lord Voldemort, an evil and powerful dark wiz...","['\n', '\nThe Magic Begins November 16th.', '\n']"
7,"[Comedy, Family, Fantasy]",Hocus Pocus혻,www.imdb.com/title/tt0107120/mediaviewer/rm354...,"[A curious youngster moves to Salem, where he ...","[Hocus Pocus begins in a small town in 1693, w...","['\n', ""\nIt's just a bunch of Hocus Pocus!""]"
8,"[Action, Adventure, Comedy, Fantasy]",Jumanji: Welcome to the Jungle혻,www.imdb.com/title/tt2283362/mediaviewer/rm193...,[Four teenagers are sucked into a magical vide...,"[In 1996, teenager Alex Vreeke (, ) receives t...","['\n', '\nLanding This Christmas.', '\n']"
9,"[Animation, Adventure, Comedy, Family]",Finding Nemo혻,www.imdb.com/title/tt0266543/mediaviewer/rm331...,[After his son is captured in the Great Barrie...,"[Two clownfish, Marlin (, ) and his wife Coral...","['\n', ""\n71% of the Earth's surface is covere..."


In [21]:
df.movie_name = df.movie_name.apply(lambda x: x.replace('혻',''))

#df.movie_tagline = df.movie_tagline.apply(lambda x: x.replace('xa0','n'))


In [22]:
df.movie_summary

0       [Four teen girls diving in a ruined underwater...
1       [Twenty-seven years after their first encounte...
2       [After a space merchant vessel perceives an un...
3                                                      []
4       [A meek Hobbit from the Shire and eight compan...
5       [A 90-year-old horticulturist and Korean War v...
6       [An orphaned boy enrolls in a school of wizard...
7       [A curious youngster moves to Salem, where he ...
8       [Four teenagers are sucked into a magical vide...
9       [After his son is captured in the Great Barrie...
10      [A fierce rebel wrestler Kichha fights brutal ...
11      [A cryptic message from 007's past sends him p...
12      [A Duke's son leads desert warriors against th...
13      [Loyalties are tested when five friends and fo...
14      [A sole survivor tells of the twisty events le...
15      [While trying to spice up their marriage in th...
16      [Based on the true story of a real-life friend...
17      [A New

In [26]:
import re
df.movie_summary = df.movie_summary.apply(lambda x: str(x).lower())
#df.movie_summary = df.movie_summary.apply(lambda x: re.sub(' {2,}', '', x))

df.movie_summary = df.movie_summary.apply(lambda x: re.sub("[^A-Za-z. 0-9]", '', x))

df.movie_synopsis = df.movie_synopsis.apply(lambda x : str(x).lower())
df.movie_synopsis = df.movie_synopsis.apply(lambda x : re.sub('[^A-Za-z. ]', '', x))

df.drop(df[df['movie_synopsis']==''].index, inplace = True)

In [24]:
df.movie_tagline = df.movie_tagline.apply(lambda x: str(x).lower())
df.movie_tagline = df.movie_tagline.apply(lambda x: x.replace('\\n',''))
df.movie_tagline = df.movie_tagline.apply(lambda x: x.replace('\\xa0',''))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub('[^A-Za-z. 0-9]', '', x))
df.movie_tagline = df.movie_tagline.apply(lambda x : re.sub(' {2,}', '', x))


,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
3,"[Horror, Mystery, Thriller]",The Conjuring 3,www.imdb.com/title/tt7069210/mediaviewer/rm175...,,,


In [35]:
index = df[df['movie_tagline']==''].index
df.drop(index, inplace = True)

summary_index = df[df['movie_summary']==''].index
index = df[df['movie_summary'].str.contains('  ')].index
#df.movie_synopsis = df.movie_synopsis.apply(lambda x : re.sub('n {2,}', '', x))
#synopsis_index = df[df['movie_synopsis']==''].index
df.drop(index , inplace = True)


df.reset_index()


,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
0,0,"[Adventure, Drama, Horror, Thriller]",47 Meters Down: Uncaged,www.imdb.com/title/tt7329656/mediaviewer/rm242...,47 meters down uncaged follows the diving adve...,the film starts at a girls school in yucatan m...,the next chapter
1,1,"[Drama, Fantasy, Horror]",It Chapter Two,www.imdb.com/title/tt7349950/mediaviewer/rm324...,twentyseven years after their first encounter ...,iderry maine a young gay man is beaten by a ho...,youll float again
2,2,"[Horror, Sci-Fi]",Alien,www.imdb.com/title/tt0078748/mediaviewer/rm299...,after a space merchant vessel perceives an unk...,the opening credits appear in front of a large...,its alien the 8th passenger.
3,4,"[Adventure, Drama, Fantasy]",The Lord of the Rings: The Fellowship of the Ring,www.imdb.com/title/tt0120737/mediaviewer/rm359...,a meek hobbit from the shire and eight compani...,the prologue spoken by galadriel shows the dar...,all we have to decide is what to do with the ...
4,5,"[Crime, Drama, Thriller]",The Mule,www.imdb.com/title/tt7959026/mediaviewer/rm320...,a 90yearold horticulturist and korean war vete...,this movie is based on a true story of a world...,nobody runs forever
5,6,"[Adventure, Family, Fantasy]",Harry Potter and the Sorcerer's Stone,www.imdb.com/title/tt0241527/mediaviewer/rm683...,an orphaned boy enrolls in a school of wizardr...,lord voldemort an evil and powerful dark wizar...,the magic begins november 16th.
6,7,"[Comedy, Family, Fantasy]",Hocus Pocus,www.imdb.com/title/tt0107120/mediaviewer/rm354...,a curious youngster moves to salem where he st...,hocus pocus begins in a small town iwere a tee...,its just a bunch of hocus pocus
7,8,"[Action, Adventure, Comedy, Fantasy]",Jumanji: Welcome to the Jungle,www.imdb.com/title/tt2283362/mediaviewer/rm193...,four teenagers are sucked into a magical video...,iteenager alex vreeke receives the original ...,landing this christmas.
8,9,"[Animation, Adventure, Comedy, Family]",Finding Nemo,www.imdb.com/title/tt0266543/mediaviewer/rm331...,after his son is captured in the great barrier...,two clownfish marliand his wife coral admire...,71 of the earths surface is covered by water....
9,12,"[Action, Adventure, Sci-Fi]",Dune,www.imdb.com/title/tt0087182/mediaviewer/rm423...,a dukes son leads desert warriors against the ...,in the opening monologue princess irulaexplain...,a place beyond your dreams. a movie beyond yo...


In [36]:
for idx, summary in enumerate(df['movie_summary']):
    x = summary.split('. ')
    #print(x)
    if len(x) <= 2:
        df['movie_summary'][idx] = summary
    else:
        df['movie_summary'][idx] = summarize(summary, word_count=30,ratio = 0.3)

W1022 23:00:25.461535  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:25.464526  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:25.558277  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:25.560271  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:25.650029  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:25.659007  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:25.785667  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:25.788658  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:26.048963  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:26.057939  8828 summarizer.py:356] Input corpus is expected to have at 

W1022 23:00:30.334502  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:30.393342  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:30.398333  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:30.562887  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:30.566879  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:30.641678  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:30.643672  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:30.718472  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:30.720466  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:30.770335  8828 summarizer.py:432] Input text is expected to have at 

W1022 23:00:33.152961  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:33.153963  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:33.158945  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:33.159942  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:33.206821  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:33.208810  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:33.287599  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:33.288596  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:33.329488  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:33.337466  8828 summarizer.py:356] Input corpus is expected to have at 

W1022 23:00:35.119699  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:35.143634  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:35.145630  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:35.181534  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:35.189512  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:35.200484  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:35.203475  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:35.214447  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:35.222433  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:35.252346  8828 summarizer.py:432] Input text is expected to have at 

W1022 23:00:36.527931  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:36.528929  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:36.533915  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:36.535913  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:36.539901  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:36.541896  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:36.546881  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:36.548876  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:36.556856  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:36.558850  8828 summarizer.py:356] Input corpus is expected to have at 

...

W1022 23:00:46.126258  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:46.135233  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:46.137228  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:46.216017  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:46.219010  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:46.230981  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:46.232972  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:46.265887  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:46.268876  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:46.335705  8828 summarizer.py:432] Input text is expected to have at 

W1022 23:00:47.043803  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:47.044799  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:47.111632  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:47.113617  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:47.181436  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:47.185425  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:47.196396  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:47.198390  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:47.235292  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:47.244270  8828 summarizer.py:356] Input corpus is expected to have at 

W1022 23:00:47.846654  8828 summarizer.py:440] Couldn't get relevant sentences.
W1022 23:00:47.881564  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:47.883557  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:47.890539  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:47.892532  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:47.913477  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:47.916468  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:47.924447  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:47.927440  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:47.937415  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W10

W1022 23:00:48.626571  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:48.629561  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:48.639534  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:48.641532  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:48.658506  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:48.664468  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:48.670454  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:48.678433  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:48.698377  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:48.700373  8828 summarizer.py:356] Input corpus is expected to have at 

W1022 23:00:49.363597  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:49.380552  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:49.382547  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:49.406504  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:49.422441  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:49.434409  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:49.441423  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:49.453358  8828 summarizer.py:432] Input text is expected to have at least 10 sentences.
W1022 23:00:49.455352  8828 summarizer.py:356] Input corpus is expected to have at least 10 documents.
W1022 23:00:49.464329  8828 summarizer.py:432] Input text is expected to have at 

In [29]:
index = df[df['movie_summary']==''].index
df.drop(index, inplace=True)
df.reset_index()

C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,index,movie_genre,movie_name,movie_poster,movie_summary,movie_synopsis,movie_tagline
0,0,"[Adventure, Drama, Horror, Thriller]",47 Meters Down: Uncaged,www.imdb.com/title/tt7329656/mediaviewer/rm242...,47 meters down uncaged follows the diving adve...,the film starts at a girls school in yucatan m...,the next chapter
1,1,"[Drama, Fantasy, Horror]",It Chapter Two,www.imdb.com/title/tt7349950/mediaviewer/rm324...,twentyseven years after their first encounter ...,in derry maine a young gay man is beaten by a...,youll float again
2,2,"[Horror, Sci-Fi]",Alien,www.imdb.com/title/tt0078748/mediaviewer/rm299...,after a space merchant vessel perceives an unk...,the opening credits appear in front of a large...,its alien the 8th passenger.
3,4,"[Adventure, Drama, Fantasy]",The Lord of the Rings: The Fellowship of the Ring,www.imdb.com/title/tt0120737/mediaviewer/rm359...,a meek hobbit from the shire and eight compani...,the prologue spoken by galadriel shows the dar...,all we have to decide is what to do with the ...
4,5,"[Crime, Drama, Thriller]",The Mule,www.imdb.com/title/tt7959026/mediaviewer/rm320...,a 90yearold horticulturist and korean war vete...,this movie is based on a true story of a world...,nobody runs forever
5,6,"[Adventure, Family, Fantasy]",Harry Potter and the Sorcerer's Stone,www.imdb.com/title/tt0241527/mediaviewer/rm683...,an orphaned boy enrolls in a school of wizardr...,lord voldemort an evil and powerful dark wizar...,the magic begins november 16th.
6,7,"[Comedy, Family, Fantasy]",Hocus Pocus,www.imdb.com/title/tt0107120/mediaviewer/rm354...,a curious youngster moves to salem where he st...,hocus pocus begins in a small town in were a ...,its just a bunch of hocus pocus
7,8,"[Action, Adventure, Comedy, Fantasy]",Jumanji: Welcome to the Jungle,www.imdb.com/title/tt2283362/mediaviewer/rm193...,four teenagers are sucked into a magical video...,in teenager alex vreeke receives the origin...,landing this christmas.
8,9,"[Animation, Adventure, Comedy, Family]",Finding Nemo,www.imdb.com/title/tt0266543/mediaviewer/rm331...,after his son is captured in the great barrier...,two clownfish marlin and his wife coral ad...,71 of the earths surface is covered by water....
9,11,"[Action, Adventure, Thriller]",Spectre,www.imdb.com/title/tt2379713/mediaviewer/rm388...,a cryptic message from 007s past sends him pit...,n n,the dead are alive


In [37]:

tl = list(df["movie_tagline"])
tl = ["".join([ j for j in i if j != "\n"]).strip() for i in tl]
#print(tl)

#summ = list(df['movie_summary'])
summ = list(df['movie_synopsis'])



In [38]:

df.movie_tagline = tl
df.movie_summary = summ

In [39]:

sources = [i.split() for i in df['movie_synopsis']]
targets = [i.split() for i in df['movie_tagline']]

In [40]:
# vocabulary for sources
s_vocab = list(set(sum(sources, [])))
s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}



In [41]:
# vocabulary for targets
t_vocab = list(set(sum(targets, [])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>', '<eos>'] + t_vocab
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}


print(len(source2idx), len(target2idx))

66768 4081


In [42]:
def preprocess(sequences, max_len, dic, mode = 'source'):
    assert mode in ['source', 'target'], 'source와 target 중에 선택해주세요.'
    
    if mode == 'source':
        # preprocessing for source (encoder)
        s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
        
        #s_input = [[x for x in each_input if x] for each_input in s_input]
        #pprint(s_input)
        s_len = list(map(lambda sentence : len(sentence), s_input))
        s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
        return s_len, s_input
    

    elif mode == 'target':
        # preprocessing for target (decoder)
        # input
        t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
        
        #t_input = [[x for x in each_input if x] for each_input in t_input]
        #pprint(t_input)
        t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
        t_len = list(map(lambda sentence : len(sentence), t_input))
        t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
     
        # output
        t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
        t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
        t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
        
        return t_len, t_input, t_output

In [43]:
# preprocessing for source
s_max_len = 10
s_len, s_input = preprocess(sequences = sources,
                            max_len = s_max_len, dic = source2idx, mode = 'source')
#print(s_len, s_input)

In [44]:
# preprocessing for target
t_max_len = 12
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_len, t_input, t_output)

[5, 5, 7, 18, 5, 7, 9, 5, 20, 12, 12, 9, 5, 6, 5, 10, 9, 9, 9, 6, 5, 11, 5, 12, 25, 8, 8, 6, 5, 6, 4, 10, 4, 8, 5, 8, 26, 17, 7, 6, 8, 13, 5, 8, 11, 6, 7, 12, 8, 13, 10, 19, 8, 9, 6, 10, 8, 15, 8, 8, 6, 6, 17, 9, 8, 9, 8, 75, 8, 35, 5, 12, 8, 8, 9, 4, 10, 10, 8, 12, 6, 12, 8, 13, 8, 8, 4, 12, 22, 4, 9, 7, 15, 6, 4, 7, 10, 23, 10, 8, 10, 8, 8, 7, 9, 6, 10, 10, 7, 8, 7, 7, 7, 6, 27, 12, 5, 8, 5, 6, 5, 12, 6, 5, 12, 6, 4, 12, 8, 6, 16, 6, 8, 8, 8, 9, 6, 5, 9, 37, 15, 6, 5, 4, 13, 13, 20, 8, 8, 11, 6, 14, 8, 7, 13, 6, 14, 9, 7, 10, 10, 7, 9, 16, 6, 11, 8, 14, 6, 10, 6, 13, 8, 13, 22, 8, 21, 11, 10, 23, 6, 6, 10, 18, 6, 10, 6, 11, 12, 5, 15, 5, 7, 5, 12, 8, 9, 10, 8, 8, 6, 4, 29, 9, 6, 19, 11, 20, 14, 6, 5, 16, 13, 5, 16, 17, 11, 13, 20, 7, 35, 29, 10, 7, 14, 13, 10, 10, 7, 7, 5, 13, 10, 7, 26, 18, 21, 7, 9, 8, 18, 11, 8, 6, 17, 8, 6, 14, 5, 22, 9, 5, 16, 7, 18, 6, 45, 9, 11, 9, 8, 7, 7, 9, 23, 13, 8, 9, 12, 13, 16, 9, 6, 6, 31, 9, 17, 4, 7, 11, 14, 5, 7, 12, 8, 9, 8, 5, 17, 14, 10, 10, 19,

# hyper-param

In [45]:
# hyper-parameters
epochs = 100
batch_size = 2
learning_rate = .005
total_step = epochs / batch_size
buffer_size = 100
n_batch = buffer_size//batch_size
embedding_dim = 32
units = 32

# input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
data = data.shuffle(buffer_size = buffer_size)
data = data.batch(batch_size = batch_size)
# s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()

In [46]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [47]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [48]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
                
    def call(self, x, hidden, enc_output):
        
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [49]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    
#     print("real: {}".format(real))
#     print("pred: {}".format(pred))
#     print("mask: {}".format(mask))
#     print("loss: {}".format(tf.reduce_mean(loss_)))
    
    return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.train.AdamOptimizer()

# creating check point (Object-based saving)
checkpoint_dir = './data_out/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                encoder=encoder,
                                decoder=decoder)

# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir=checkpoint_dir)

W1022 23:05:06.655924  8828 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [50]:
EPOCHS = 100

for epoch in range(EPOCHS):
    
    hidden = encoder.initialize_hidden_state()
    #print(hidden.shape)
    total_loss = 0
    for i, (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
        loss = 0
        #print(s_input, s_input.shape)
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(s_input, hidden)

            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
            
            #Teacher Forcing: feeding the target as the next input
            for t in range(1, t_input.shape[1]):
                
                predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(t_input[:, t], predictions)
            
                dec_input = tf.expand_dims(t_input[:, t], 1) #using teacher forcing
                
        batch_loss = (loss / int(t_input.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradient = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradient, variables))
        
    if epoch % 10 == 0:
        #save model every 10 epoch
        print('Epoch {} Loss {:.4f} Batch Loss {:.4f}'.format(epoch,
                                            total_loss / n_batch,
                                            batch_loss.numpy()))
        checkpoint.save(file_prefix = checkpoint_prefix)

ValueError: Tensor's shape (10, 2, 32) is not compatible with supplied shape [Dimension(10), Dimension(1), 32]

In [0]:
#restore checkpoint

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
#sentence = 'I feel hungry'
sentence = 'four teen girls diving in a ruined underwater city quickly learn theyve entered the territory of the deadliest shark species in the claustrophobic labyrinth of submerged caves. 47 meters down uncaged follows the diving adventure of four teenage girls sophie nlisse corinne foxx brianne tju and sistine stallone exploring a submerged mayan city. once inside their rush of excitement turns into a jolt of terror as they discover the sunken ruins are a hunting ground for deadly great white sharks. with their air supply steadily dwindling the friends must navigate the underwater labyrinth of claustrophobic caves and eerie tunnels in search of a way out of their watery hell.'

In [2]:
df.movie_tagline[2]

NameError: ignored

In [0]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    inputs = [inp_lang[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
        
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
        
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
   
    for t in range(max_length_targ):
        

        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '
        
        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)    
            
    return result, sentence
    
result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

print(sentence)
print(result)
33333333333333333333333333333333333333333333333332print(output_sentence)

four teen girls diving in a ruined underwater city quickly learn theyve entered the territory of the deadliest shark species in the claustrophobic labyrinth of submerged caves. 47 meters down uncaged follows the diving adventure of four teenage girls sophie nlisse corinne foxx brianne tju and sistine stallone exploring a submerged mayan city. once inside their rush of excitement turns into a jolt of terror as they discover the sunken ruins are a hunting ground for deadly great white sharks. with their air supply steadily dwindling the friends must navigate the underwater labyrinth of claustrophobic caves and eerie tunnels in search of a way out of their watery hell.
they were watch out <eos> 
four teen girls diving in a ruined underwater city quickly learn theyve entered the territory of the deadliest shark species in the claustrophobic labyrinth of submerged caves. 47 meters down uncaged follows the diving adventure of four teenage girls sophie nlisse corinne foxx brianne tju and sist

In [0]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    inputs = [inp_lang[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
        
    result = ''
    
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    idx = 0
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
    
    xList = []
    for t in range(max_length_targ):
      predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
      #print("*"*100)
      #print(predictions)
      #print("*"*100)
      print(tf.argmax(predictions[0]))
      x = tf.nn.top_k(predictions[0], k=5, sorted=True, name=None)
      for i in range(5):
        predicted_id = x[1][i].numpy()


        result += idx2target[predicted_id] + ' '

        if idx2target.get(predicted_id) == '<eos>':
#             return result, sentence
          xList.append([result, sentence])
          result=""

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)  

    return xList

# result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
tList=prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
# print(sentence)
from pprint import pprint
pprint(tList)

In [0]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
#     sentence = preprocess_sentence(sentence)

    inputs = [inp_lang[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang['<bos>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += idx2target[predicted_id] + ' '

        if idx2target.get(predicted_id) == '<eos>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# 새 섹션

# 새 섹션

In [0]:
sentence = 'I feel hungry'


In [0]:
result, sentence, a = evaluate(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)
print(result)
print(sentence)

ValueError: ignored